# Scrape URLs And Store The Cleaned Text In Silver Table

In [0]:
#!pip install newspaper3
#!pip install -e https://github.com/codelucas/newspaper/archive/master.zip
%pip install -e git+https://github.com/codelucas/newspaper.git@master#egg=newspaper3k

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
Obtaining newspaper3k from git+https://github.com/codelucas/newspaper.git@master#egg=newspaper3k
  Cloning https://github.com/codelucas/newspaper.git (to revision master) to /local_disk0/.ephemeral_nfs/envs/pythonEnv-1fdbdb46-3dfd-4898-819b-14a88ed6e6f7/src/newspaper3k
  Running command git clone --filter=blob:none --quiet https://github.com/codelucas/newspaper.git /local_disk0/.ephemeral_nfs/envs/pythonEnv-1fdbdb46-3dfd-4898-819b-14a88ed6e6f7/src/newspaper3k
  Resolved https://github.com/codelucas/newspaper.git to commit f622011177f6c2e95e48d6076561e21c016f08c3
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [0]:
%pip install beautifulsoup4

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
%pip install nltk

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
DEPRECATION: distro-info 1.1build1 has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of distro-info or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.


In [0]:
dbutils.library.restartPython()

In [0]:
%sql
--drop table infosys.kamikaze_rtsa.silver_table_processed

In [0]:
%sql
/*
CREATE TABLE infosys.kamikaze_rtsa.silver_table_processed (
  SNo integer,
  batch integer,
  Timestamp timestamp,
  url string,
  article_title string,
  article_text string,
  transformed_text string
)USING DELTA
--location '/FileStore/tables/silver_table';
*/

In [0]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, TimestampType, ShortType, DataType
from newspaper import Article
import nltk
import re
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
spark = SparkSession.builder.appName("RTSA_app").getOrCreate()

In [0]:
def getArticle(url):
    try:
        processed = 0
        article = Article(url)
        article.download()
        ##Article was downloaded
        if article.download_state == 2:
            processed = 1
            article.parse()
    except article.download_exception_msg as e:
        if article.download_state==1:
            print(article.download_exception_msg)
        else:
            raise e
    return(article.title,article.text,processed,article.download_state)

In [0]:
def insert_into_silver():
    out = spark.sql("""
                    INSERT INTO infosys.kamikaze_rtsa.silver_table_processed
                    SELECT distinct SNo, batch, CURRENT_TIMESTAMP() AS Timestamp, urls, title, text, clean_txt
                    FROM   temp_silver
                    WHERE  title != '' 
                    AND    text  != '') 
                    """)
    for row in out.collect():
        if(row[0])>0 : 
            print("Successfully inserted {} records".format(row[0]))
    return(out)

In [0]:
def update_bronze():
    out = spark.sql("""
                    UPDATE infosys.kamikaze_rtsa.bronze_table_raw t1
                    SET processed = (SELECT max(processed) FROM temp_bronze WHERE urls = t1.url),
                        status = (SELECT max(status) FROM temp_bronze WHERE urls = t1.url)
                    WHERE EXISTS (SELECT 1 
                                FROM  temp_bronze t2
                                WHERE t2.urls = t1.url)
                    """)
    return(out)

In [0]:
def update_msg(out):
    for row in out.collect():
        if(row[0])>0 : 
            print("Successfully updated {} records".format(row[0]))
        else:
            print("No updates performed")

In [0]:
def remove_url(input_text):
    pattern=r'(?i)\b((?:[a-z][\w-]+:(?:/{1,3}|[a-z0-9%])|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:\'".,<>?«»“”‘’]))';
    match = re.findall(pattern, input_text)
    for m in match:
        url = m[0]
        input_text = input_text.replace(url, '')
    return(input_text)

In [0]:
## Insert records in to Silver table and update bronze table for processed records
def process_urls(df):
    #bronze_table = spark.sql("""SELECT * FROM infosys.kamikaze_rtsa.bronze_table_raw""")

    #df = bronze_table.select("url").filter((bronze_table["processed"] == 0))
    #df = df.limit(50)

    urls = [row[0] for row in df.collect()]
    title, text, clean_txt, processed, status = [], [], [], [], []
    for url in urls:
        tit, txt, procd,stat  = getArticle(url)
        title.append(tit)
        text.append(txt)
        clean_txt.append(remove_url(txt))
        processed.append(procd)
        status.append(stat)
    data = list(zip(urls, title, text, clean_txt))
    data1 = list(zip(urls, processed, status))
    df = spark.createDataFrame(data, ["urls", "title", "text", "clean_txt"])
    df1 = spark.createDataFrame(data1, ["urls", "processed", "status"])
    
    current_max_batch = spark.sql("""SELECT MAX(batch) AS max_batch FROM infosys.kamikaze_rtsa.silver_table_processed""").first().max_batch
    next_batch = current_max_batch + 1 if current_max_batch is not None else 1
    df = df.withColumns({"SNo"  : F.monotonically_increasing_id()+1, 
                        "batch": F.lit(next_batch)})
    df = df.withColumn("SNo",df.SNo.cast('integer'))

    df.createOrReplaceTempView('temp_silver')
    df1.createOrReplaceTempView('temp_bronze')
    
    out = insert_into_silver()
    #update_msg(out)
    out = update_bronze()
    update_msg(out)

In [0]:
def load_data():
    bronze_table = spark.sql("""SELECT * FROM infosys.kamikaze_rtsa.bronze_table_raw""")
    df = bronze_table.select("url").filter((bronze_table["processed"] == 0) & ((bronze_table["status"] == 0)))
    chunks = chunkify(df,500)
    return(chunks)

In [0]:
def chunkify(df: F.DataFrame, chunk_size: int):
    start = 0
    length = df.count()

    # If DF is smaller than the chunk, return the DF
    if length <= chunk_size:
        yield df
        return

    # Yield individual chunks
    while start + chunk_size <= length:
        yield df.take(chunk_size)
        start = start + chunk_size

    # Yield the remainder chunk, if needed
    if start < length:
        yield df.take(length - start)

In [0]:
def main():
    spark = SparkSession.builder.appName("RTSA_app").getOrCreate()
    chunks = load_data()
    for chunk in chunks:
        df = spark.createDataFrame(data=chunk)
        process_urls(df)

In [0]:
if __name__ == "__main__":
    main()

Building prefix dict from /local_disk0/.ephemeral_nfs/envs/pythonEnv-1fdbdb46-3dfd-4898-819b-14a88ed6e6f7/lib/python3.10/site-packages/jieba/dict.txt ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 2.150599479675293 seconds.
Prefix dict has been built succesfully.


Successfully inserted 471 records
Successfully updated 500 records


/databricks/python/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/databricks/python/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname CDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/databricks/python/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {t

Successfully inserted 471 records
Successfully updated 500 records
Successfully inserted 473 records
Successfully updated 500 records
Successfully inserted 470 records
Successfully updated 500 records


/databricks/python/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname MST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


...

Successfully inserted 448 records
Successfully updated 500 records
Successfully inserted 454 records
Successfully updated 500 records
Successfully inserted 450 records
Successfully updated 500 records
Successfully inserted 466 records
Successfully updated 500 records
Successfully inserted 460 records
Successfully updated 500 records
Successfully inserted 459 records
Successfully updated 500 records
Successfully inserted 474 records
Successfully updated 500 records
Successfully inserted 453 records
Successfully updated 500 records
Successfully inserted 468 records
Successfully updated 500 records


/databricks/python/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


Successfully inserted 473 records
Successfully updated 500 records
Successfully inserted 458 records
Successfully updated 500 records
Successfully inserted 472 records
Successfully updated 500 records
Successfully inserted 477 records
Successfully updated 500 records


/databricks/python/lib/python3.10/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname MDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "


Successfully inserted 458 records
Successfully updated 500 records
Successfully inserted 456 records
Successfully updated 500 records
Successfully inserted 141 records
Successfully updated 154 records


In [0]:
%sql
--select count(*) from infosys.kamikaze_rtsa.silver_table_processed

In [0]:
%sql
/*
select processed, status, count(1) 
from infosys.kamikaze_rtsa.bronze_table_raw
group by processed, status
*/


processed status count(1) 1 2 36717 0 0 15531 0 1 2561

In [0]:
%sql
--select count(*) from infosys.kamikaze_rtsa.gold_table
